# 사진에서 정보추출(날짜정보)

In [6]:
from PIL import Image
from PIL.ExifTags import TAGS

img_path = 'C:/Workspace/python/빅데이터 지능형서비스 개발 팀프로젝트/Final Project/Data/IC_Data/test_images/sea2.jpg'
image = Image.open(img_path)
info = image._getexif();
image.close()

In [21]:
taglabel = dict()

for tag, value in info.items():
    decoded = TAGS.get(tag, tag)
    taglabel[decoded] = value

# print(taglabel)
print('DateTimeOriginal' in taglabel.keys())
'DateTime' in taglabel.keys()

True


True

In [23]:
print(taglabel['DateTime'])

2021:05:02 09:50:04


In [24]:
date = taglabel['DateTime']
date_list = date.split()
ymd = date_list[0]
date = '날짜 : ' + '. '.join(ymd.split(':'))
date

'날짜 : 2021. 05. 02'

# 사진에서 정보추출(위치정보)

In [25]:
taglabel['GPSInfo']

{1: 'N',
 2: (38.0, 11.0, 32.43),
 3: 'E',
 4: (128.0, 36.0, 15.3),
 5: b'\x00',
 6: 1.0,
 29: '2021:05:02'}

In [27]:
import numpy as np
latDeg = taglabel['GPSInfo'][2][0]
latMin = taglabel['GPSInfo'][2][1]
latSec = taglabel['GPSInfo'][2][2]
# latDeg, latMin, latSec
lonDeg = taglabel['GPSInfo'][4][0]
lonMin = taglabel['GPSInfo'][4][1]
lonSec = taglabel['GPSInfo'][4][2]
# lonDeg, lonMin, lonSec
x = (int(latDeg)+(int(latMin)/60)+(int(latSec)/3600))
y = int(lonDeg)+(int(lonMin)/60)+(int(lonSec)/3600)
Lat = f'''{str(int(latDeg))}°{str(int(latMin))}'{str(latSec)}"{taglabel["GPSInfo"][1]}'''
Lon = str(int(lonDeg)) + "°" + (str(int(lonMin)) + "'") + (str(lonSec) + '"') + taglabel['GPSInfo'][3]

In [29]:
# key 호출
filename= 'C:/Workspace/python/빅데이터 지능형서비스 개발 팀프로젝트/Final Project/Data/mapkey.txt'

with open(filename) as f:
  api_key = f.read()
client_id = api_key.split('\n')[0]
client_secret = api_key.split('\n')[1]
len(api_key)

51

In [30]:
# 위치정보 추출(api 활용)
import requests

url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={y},{x}&sourcecrs=epsg:4326&orders=legalcode&output=json&orders=addr,admcode,roadaddr"

payload={}
headers = {
  'X-NCP-APIGW-API-KEY-ID': client_id,
  'X-NCP-APIGW-API-KEY': client_secret
}

response = requests.get(url, headers=headers, data=payload).json()
print(response)

{'status': {'code': 0, 'name': 'ok', 'message': 'done'}, 'results': [{'name': 'legalcode', 'code': {'id': '4221010800', 'type': 'L', 'mappingId': '01210108'}, 'region': {'area0': {'name': 'kr', 'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}, 'area1': {'name': '강원도', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 128.3115261, 'y': 37.8603672}}, 'alias': '강원'}, 'area2': {'name': '속초시', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 128.59184, 'y': 38.207169}}}, 'area3': {'name': '조양동', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 128.590788, 'y': 38.187618}}}, 'area4': {'name': '', 'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}}]}


In [33]:
response['results'][0]['region']['area1']['name'] + ' ' + response['results'][0]['region']['area2']['name']

'강원도 속초시'

# 사진에서 정보추출(기상정보)

In [47]:
filename= 'C:/Workspace/python/빅데이터 지능형서비스 개발 팀프로젝트/Final Project/Data/areacode.csv'
ac = pd.read_csv(filename)
sido_list = list(ac['지점명'].copy())
ac

,지점,지점명,관리관서
0,90,속초,강원지방기상청
1,93,북춘천,춘천기상대
2,95,철원,강원지방기상청
3,98,동두천,수도권기상청
4,99,파주,수도권기상청
...,...,...,...
90,285,합천,울산기상대
91,288,밀양,울산기상대
92,289,산청,창원기상대
93,294,거제,부산지방기상청


In [65]:
import requests
import json
import pandas as pd

# key 암호화
filename= 'C:/Workspace/python/빅데이터 지능형서비스 개발 팀프로젝트/Final Project/Data/weather.txt'

with open(filename) as f:
  api_key = f.read()
len(api_key)


filename= 'C:/Workspace/python/빅데이터 지능형서비스 개발 팀프로젝트/Final Project/Data/areacode.csv'
ac = pd.read_csv(filename)

# 날짜 지정
startDt = '20211205' 
si = '속초시'
# 지역 코드 선택

for location in ac.지점명:
    if location in si:
         area = ac[ac.지점명 == location].지점

stnIds = area
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params ={'serviceKey' : api_key, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'json', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : startDt, 'endDt' : startDt, 'stnIds' : stnIds }

response = requests.get(url, params=params).json()
print(response)

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'}, 'body': {'dataType': 'JSON', 'items': {'item': [{'stnId': '90', 'stnNm': '속초', 'tm': '2021-12-05', 'avgTa': '4.7', 'minTa': '-0.2', 'minTaHrmt': '532', 'maxTa': '12.0', 'maxTaHrmt': '1425', 'mi10MaxRn': '', 'mi10MaxRnHrmt': '', 'hr1MaxRn': '', 'hr1MaxRnHrmt': '', 'sumRnDur': '', 'sumRn': '', 'maxInsWs': '4.8', 'maxInsWsWd': '110', 'maxInsWsHrmt': '1321', 'maxWs': '3.0', 'maxWsWd': '140', 'maxWsHrmt': '1225', 'avgWs': '1.5', 'hr24SumRws': '1338', 'maxWd': '270', 'avgTd': '-6.5', 'minRhm': '29', 'minRhmHrmt': '1450', 'avgRhm': '44.4', 'avgPv': '3.8', 'avgPa': '1026.0', 'maxPs': '1030.6', 'maxPsHrmt': '1002', 'minPs': '1026.4', 'minPsHrmt': '2340', 'avgPs': '1028.2', 'ssDur': '9.6', 'sumSsHr': '8.9', 'hr1MaxIcsrHrmt': '', 'hr1MaxIcsr': '', 'sumGsr': '', 'ddMefs': '', 'ddMefsHrmt': '', 'ddMes': '', 'ddMesHrmt': '', 'sumDpthFhsc': '', 'avgTca': '0.1', 'avgLmac': '0.0', 'avgTs': '3.8', 'minTg': '-2.3', 'avgCm5Te': '

In [74]:
# ['avgRhm', 'avgTs', 'sumRn', 'ddMes']
response['response']['body']['items']['item'][0].keys()
response['response']['body']['items']['item'][0]['avgRhm']

'44.4'